# ЦЕЛЬ ЛАБОРАТОРНОЙ РАБОТЫ
Целью этой задачи было показать работу MPI_SEND, MPI_Send, MPI_Send,
MPI_Rsend) и завершить анализ производительности каждого из них.

# ПОСТАНОВКА ЗАДАЧИ
Мы разрабатываем простое приложение, которое отправляет несколько байт данных из одного процесса в другой
один.

# КРАТКАЯ ТЕОРИЯ

Проект Open MPI - это реализация интерфейса передачи сообщений с открытым исходным кодом, которая
разработан и поддерживается консорциумом научных, исследовательских и промышленных партнеров. Таким образом, Open MPI может объединить опыт, технологии и ресурсы всего высокопроизводительного вычислительного сообщества, чтобы создать лучшую доступную библиотеку MPI. Open MPI предлагает преимущества для поставщиков систем и программного обеспечения, разработчиков приложений и исследователей компьютерных наук.

MPI имеет несколько различных "режимов отправки." Они представляют собой различные варианты буферизации (где хранятся данные до их получения) и синхронизации (когда отправка завершена). В следующем примере я использую "отправить буфер" для пользовательского буфера для отправки. 

- MPI_Send
  MPI_Send не завершится, пока вы не сможете использовать буфер отправки. Он может или не может блокировать (разрешено буферизовать либо на стороне отправителя или получателя, либо ждать соответствующего приема).
- MPI_Bsend 
  May buffer; завершается сразу, и вы можете использовать буфер отправки. Появилось позднее дополнение к спецификации MPI. Следует использовать только в случае крайней необходимости. 
- MPI_Ssend
  не завершится, пока сопоставление не будет отправлено
- MPI_Rsend
  Может использоваться, только если совпадение уже учтено. Пользователь отвечает за написание правильной программы.
- MPI_Isend
   Неблокирующая отправка. Но не обязательно асинхронный. Вы не можете повторно использовать буфер отправки до тех пор, пока либо успешно, вы ждете, что сообщение было получено (см. MPI_Request_free). Заметим также, что, хотя I относится к immediate, на MPI_Isend нет требований к производительности. Немедленная отправка должна быть возвращена пользователю без необходимости соответствующего получения в месте назначения.
   
 Реализация может свободно отправлять данные в пункт назначения перед return, если вызов send не блокирует ожидание соответствующего приема. Отличающийся
стратегии отправки данных предлагают различные преимущества и недостатки производительности, которые будут зависеть от приложения
- Mpi_ibsend буферизованный неблокирующий
- MPI_Issend синхронный неблокирующий. Обратите внимание, что ожидание завершится только после успешного получения.
- MPI_Irsend как с MPI_Rsend, но неблокирующий.

Обратите внимание, что "неблокирующий" относится только к тому, доступен ли буфер данных для повторного использования после вызова. Например, никакая часть спецификации MPI не предусматривает одновременной работы передачи данных и вычислений.

# Алгоритм реализации

In [1]:
%cat hello.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

#include <unistd.h>
#include <stdbool.h>

#ifndef MSG_LEN
# define MSG_LEN 32
#endif


#ifndef SEND_FN
# define SEND_FN MPI_Send
#endif

#if !defined(SYNC) && !defined(SEND_RECV) && !defined(ASYNC)
# define SYNC
#endif



void rand_str(char *str, size_t len) 
{
	for(size_t i = 0; i < len - 1; ++i) {
		str[i] = rand() % 26 + 64;
	}
	str[len] = 0;
}

int main(int argc,char **argv)
{
	int rank, size;
	MPI_Init(&argc,&argv);
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);
	MPI_Comm_size(MPI_COMM_WORLD,&size);

	MPI_Request req;
	MPI_Status status;
	bool wait = false;
	srand(rank+10);

	char buf[MSG_LEN], rbuf[MSG_LEN];

#ifdef SYNC
	printf("SYNC\n");
#endif

#ifdef SEND_RECV
	printf("SEND_RECV\n");
#endif

#ifdef ASYNC
	printf("ASYNC\n");
#endif

	for(size_t i = 0; i < 10; ++i) {
			
#ifdef SYNC
		if( (i + rank) % 2 == 0 ) {
			MPI_Recv(buf, MSG_LEN, MPI_CHAR, !rank, 0, MPI_COMM_WORLD,MPI_STATUS_IGNORE);
			printf("%*sRECV(%d) : %s\n",rank*44,

# Результаты

In [2]:
import subprocess
import os


def compile(*defs, **defskw):
    args = [f"-D{k}" for k in defs] + [f"-D{k}={v}" for k, v in defskw.items()]
    _cmd = 'mpicc -o hello hello.c'.split() + args
    # print(' '.join(_cmd))
    cmd = subprocess.run(_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  
    if(cmd.stdout): print('cmd.stdout', cmd.stdout)
    if(cmd.stderr): print('cmd.stderr', cmd.stderr)
    
def run(env=None):
    cmd = subprocess.run('mpiexec -np 2 ./hello'.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    if(cmd.stderr): print('cmd.stderr', cmd.stderr)
    


## Длина сообщения

Давайте проанализируем влияние длины сообщения на производительность приложения.

In [3]:

for i in range(8):
    compile(MSG_LEN=10**i)
    print(f"Using message length {10**i}", end="\n\t")
    %timeit run()
    print()


Using message length 1
	11.1 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using message length 10
	11.1 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using message length 100
	11.3 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using message length 1000
	11.1 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using message length 10000
	13.3 ms ± 481 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using message length 100000
	27.6 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Using message length 1000000
	166 ms ± 6.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Using message length 10000000
	5.46 ms ± 260 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



## МПЦ отправка метод

Проанализируем влияние метода send на производительность приложения.

In [4]:
for snd in 'MPI_Rsend MPI_Ssend MPI_Send'.split():

    compile(MSG_LEN=10**8, SEND_FN=snd)
    
    print(f"Using {snd} as send function", end="\n\t")
    %timeit run()
    print()


Using MPI_Rsend as send function
	5.37 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using MPI_Ssend as send function
	5.72 ms ± 378 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using MPI_Send as send function
	5.66 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



## SYNC vs ASYNC vs SENDRECV 
Позволяет анализировать влияние синхронизации на производительность и отправлять сообщения recv.

In [5]:

for snd in 'SYNC ASYNC SEND_RECV'.split():
    
    compile(snd, MSG_LEN=10**8)
    
    print(f"Using {snd}", end="\n\t")
    %timeit run()
    print()
    


Using SYNC
	5.94 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using ASYNC
	5.77 ms ± 90.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Using SEND_RECV
	5.91 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



# ВЫВОД

Лучшая производительность, вероятно, если вы можете написать свою программу, то вы могли использовать только MPI_Ssend для больших данных, а для маленьких MPI_Send работает лучше, потому что для больших данных MPI_Ssend может полностью избежать буферизации данных. В то время как MPI_Send позволяет реализации MPI максимальную гибкость в выборе способа доставки данных. Можно использовать MPI_Bsend только тогда, когда слишком неудобно использовать MPI_Isend, поскольку MPI_Bsend немедленно возвращает буфер. Остальные подпрограммы MPI_Send, MPI_Issend и т. д., редко используются, но могут иметь значение при написании зависящего от системы кода передачи сообщений полностью в MPI.